In [ ]:
import os
import tensorflow as tf
import numpy as np
from google.colab import drive
#set seed
SEED=1234
tf.random.set_seed(SEED)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cwd='/content/drive/My Drive/Kaggle1'

from tensorflow.keras.preprocessing.image import ImageDataGenerator


apply_data_augmentation = True

# Create training ImageDataGenerator object
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=10,
                                        width_shift_range=10,
                                        height_shift_range=10,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='constant',
                                        cval=0,
                                        rescale=1./255,
                                        validation_split=0.2)
else:
    train_data_gen = ImageDataGenerator(rescale=1./255,
                        validation_split=0.2) # set validation split

valid_data_gen=ImageDataGenerator(rescale=1/255.,validation_split=0.2)

In [ ]:
dataset_dir='/content/drive/My Drive/Kaggle1/MaskDataset'

train_dir=os.path.join(dataset_dir,'training')
test_dir=os.path.join(dataset_dir,'test')

bs=8

# img shape
train_generator = train_data_gen.flow_from_directory(
    train_dir,
    batch_size=bs,
    class_mode='categorical', # targets are directly converted into one-hot vectors
    shuffle=True,seed=SEED,
    subset='training') # set as training data

validation_generator = valid_data_gen.flow_from_directory(
    train_dir, # same directory as training data
    batch_size=bs,
    shuffle=False,
    class_mode='categorical',
    subset='validation') # set as validation data




num_classes=len(train_generator.class_indices)

train_dataset=tf.data.Dataset.from_generator(lambda:train_generator,output_types=(tf.float32,tf.float32),output_shapes=([None,img_h,img_w,3],[None,num_classes]))

train_dataset = train_dataset.repeat()


valid_dataset=tf.data.Dataset.from_generator(lambda:validation_generator,output_types=(tf.float32,tf.float32),output_shapes=([None,img_h,img_w,3],[None,num_classes]))

valid_dataset=valid_dataset.repeat()




Found 4492 images belonging to 3 classes.
Found 1122 images belonging to 3 classes.


In [ ]:
import collections
collections.Counter(train_generator.classes)
collections.Counter(validation_generator.classes)


Counter({0: 1520, 1: 1518, 2: 1454})

Counter({0: 380, 1: 379, 2: 363})

In [ ]:
# Architecture: Features extraction -> Classifier
start_f=8 
depth=4
img_h=128
img_w=128
model=tf.keras.Sequential()
for i in range(1):
  if i==0:
    input_shape=[img_h,img_w,3]
  else:
    input_shape=[None]

    # Conv block: Conv2D -> Activation -> Pooling
  model.add(tf.keras.layers.Conv2D(filters=start_f,kernel_size=(3,3),strides=(1,1),padding='same',input_shape=input_shape))
  model.add(tf.keras.layers.ReLU())
  model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
  start_f*=2 #raddoppio i filtri ripetto al conv layer precedente

#parte fully connected----> classifier
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=256,activation='relu'))
model.add(tf.keras.layers.Dense(units=64,activation='relu'))
model.add(tf.keras.layers.Dense(units=3,activation='softmax'))

loss=tf.keras.losses.CategoricalCrossentropy()

lr=4e-4;

optimizer=tf.keras.optimizers.Adam(learning_rate=lr)
metrics=['accuracy']
model.compile(loss=loss,optimizer=optimizer,metrics=metrics)

# Visualize created model 
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 8)       224       
_________________________________________________________________
re_lu_1 (ReLU)               (None, 128, 128, 8)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 8)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               8388864   
_________________________________________________________________
dense_3 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_4 (Dense)              (None, 3)                

In [ ]:
#callbacks
import os
from datetime import datetime


exps_dir = os.path.join(cwd, 'classification_experiments_/Basic')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name='BasicCNN'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks=[]

tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)

tb_callback=tf.keras.callbacks.TensorBoard(log_dir=tb_dir,histogram_freq=1)
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)



In [ ]:
# Model Training
# --------------
model.fit(x=train_dataset,
          epochs=75  #### set repeat in training dataset
          ,steps_per_epoch=len(train_generator),
          validation_data=valid_dataset,
          validation_steps=len(validation_generator),
          callbacks=callbacks)

In [ ]:

from PIL import Image

final_path = os.path.join(dataset_dir, 'test')
image_filenames = next(os.walk(final_path))[2]  # names of the images

results={}
for image_name in image_filenames:
  
   img=Image.open(final_path+'/'+image_name).convert('RGB')
   img=img.resize((img_h,img_w))
   img_array=np.array(img)
   img_array=np.expand_dims(img_array, 0)
   img_array=np.true_divide(img_array,255)
   prediction=np.argmax(model.predict(img_array))
   results[image_name]=prediction


def create_csv(results, results_dir='./'):

    csv_fname = 'BasicCNNresults_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')
create_csv(results,cwd)

In [ ]:
# Print Confusion Matrix and Classification Report (Precision, Recall, and F1-score)
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

Y_prediction = model.predict_generator(validation_generator,len(validation_generator))
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_prediction,axis=1) 
# Convert validation observations to one hot vectors
Y_true = validation_generator.classes
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
class_report = classification_report(Y_true, Y_pred_classes, 
                                     target_names=validation_generator.class_indices.keys())  # target_names must be ordered depending on the class labels
print('Confusion Matrix:')
print(confusion_mtx)
print()
print('Classification Report:')
print(class_report)

Instructions for updating:
Please use Model.predict, which supports generators.
Confusion Matrix:
[[363   3  14]
 [ 12 293  74]
 [ 65 109 189]]

Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.96      0.89       380
           1       0.72      0.77      0.75       379
           2       0.68      0.52      0.59       363

    accuracy                           0.75      1122
   macro avg       0.74      0.75      0.74      1122
weighted avg       0.74      0.75      0.74      1122

